# 입문자를 위한, 파이썬/R 데이터 분석      

]

##  (공공데이터분석) 국토부 아파트매매 실거래가 API[data.go.kr] 연동

#### XML 국토교통부_아파트매매 실거래자료 : https://www.data.go.kr/data/15058747/openapi.do

#### 행정표준관리시스템 : https://www.code.go.kr/index.do ( [메뉴] 자주이용하는 코드 > 법정동 > [법정동코드전체자료] )

]

### 1. Import Packages

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 2. Data Loading 

#### : API Key Setting

In [ ]:
f=open("data/my_api_key.txt","r")
lines=f.readlines()
my_key=lines[0]
f.close()

In [ ]:
my_key

#### : Pre-defined

In [ ]:
#"http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade
#        ?
#        LAWD_CD=11110
#        &
#        DEAL_YMD=201512
#        &
#        serviceKey=서비스키"

In [ ]:
base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

In [ ]:
area_cd = "LAWD_CD=" + "11680"    # 11680 : 강남구
deal_dt = "DEAL_YMD=" + "202306"
svc_key = "serviceKey=" + my_key

In [ ]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

#### : Request Test

In [ ]:
response = requests.get(url)
response

In [ ]:
response.text

### Addtional Preparation : 

#### -1)  Input Data : lawd_cd / 법정동 코드(지역코드) 가공

In [ ]:
lawd_cd = pd.read_csv("data/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'
lawd_cd

In [ ]:
# Column 이름 변경 : 데이터 조작 편의성을 위함
print(lawd_cd.columns)

In [ ]:
lawd_cd.columns = ["code", "name", "check"]
lawd_cd

In [ ]:
lawd_cd.info()

In [ ]:
in_name = "종로구"

In [ ]:
lawd_cd['name'].str.contains(in_name)

In [ ]:
lawd_cd['check'] == "존재"

In [ ]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")]

In [ ]:
tmp = lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1)
tmp

In [ ]:
tmp.loc[1,'code']

In [ ]:
tmp.iloc[0,0]

In [ ]:
tmp = tmp.iloc[0,0]
#str(tmp)
tmp = tmp.astype(str)
tmp[0:5]

In [ ]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

In [ ]:
# Function 작성 :
def find_lawd_cd(in_text):
    return lawd_cd[(lawd_cd['name'].str.contains(in_text)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]    

In [ ]:
in_name = "구로구"
find_lawd_cd(in_name)

#### -2)  Input Data : lawd_cd, deal_ymd 입력받기

In [ ]:
in_lawd = input("검색지역 입력 : ")

In [ ]:
in_lawd

In [ ]:
in_deal_ymd = input("계약월 입력(예: 201912) : ")

In [ ]:
in_deal_ymd

In [ ]:
in_svckey = my_key

In [ ]:
in_lawd_cd = find_lawd_cd(in_lawd)
in_lawd_cd

In [ ]:
area_cd = "LAWD_CD=" + in_lawd_cd
deal_dt = "DEAL_YMD=" + in_deal_ymd
svc_key = "serviceKey=" + in_svckey

In [ ]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

In [ ]:
response = requests.get(url)
response

In [ ]:
response.text

### 3. Data Preprocessng 

#### : web data / API output

In [ ]:
soup = BeautifulSoup(response.text, "lxml")
soup

In [ ]:
soup.find_all('item')

In [ ]:
item = soup.find('item')

In [ ]:
#item.find('거래금액').text

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
response#.content#.text

In [ ]:
root = ET.fromstring(response.content)
root

In [ ]:
item_list = []

In [ ]:
root.find('body').find('items')[0]

In [ ]:
type(root.find('body').find('items'))

In [ ]:
child = root.find('body').find('items')[0]
child.findall('*')#[0]

In [ ]:
element = child.findall('*')[0]

In [ ]:
element.tag.strip()

In [ ]:
element.text.strip()

In [ ]:
data = {}

In [ ]:
tag = element.tag.strip()
text = element.text.strip()
data[tag] = text

In [ ]:
data

In [ ]:
def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [ ]:
items_list = find_xml_items(response)
items_list

### 4. Result : Display & Save as file 

In [ ]:
result = pd.DataFrame(items_list)

In [ ]:
result

In [ ]:
result.shape

In [ ]:
result.head()

#### : file name

In [ ]:
name_info = in_lawd + "_" + in_deal_ymd
name_info

In [ ]:
file_name = f"data/result_{name_info}.csv"
file_name

In [ ]:
result.to_csv(file_name, index=False)

## Appendix : Function 

In [ ]:
def deal_by_lawd(cd, ymd):
    
    svckey = my_key
    
    area_cd = "LAWD_CD=" + cd
    deal_dt = "DEAL_YMD=" + ymd
    svc_key = "serviceKey=" + svckey
    url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key

    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = in_lawd + "_" + in_deal_ymd
    file_name = f"data/result_{name_info}.csv"

    result.to_csv(file_name, index=False)
    
    return file_name

In [ ]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
in_lawd = input("검색지역 입력 : ")
print(in_lawd)

in_lawd_cd = find_lawd_cd(in_lawd)
print(in_lawd_cd)

in_deal_ymd = input("계약월 입력(예: 201911) : ")
print(in_deal_ymd)

In [ ]:
deal_by_lawd(in_lawd_cd, in_deal_ymd)